In [ ]:
from langchain.memory import ConversationBufferMemory

# return_messages=False가 Default Value
# True 설정 시 Chat형식 meesage로 저장해줌
# False 설정 시 문자열로 저장
memory= ConversationBufferMemory(return_messages=False)

# api에 따라 딕셔너리로 넣어줘야 함({})
memory.save_context({"input":"Hi"},{"output":"How are you?"})

memory.load_memory_variables({})
# True: {'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}
# False: {'history': 'Human: Hi\nAI: How are you?'}

{'history': 'Human: Hi\nAI: How are you?'}